In [1]:
!zenml init

⠋ Initializing ZenML repository at e:\GitHub\ML_OPS.

⠙ Initializing ZenML repository at e:\GitHub\ML_OPS.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.

ZenML repository initialized at e:\GitHub\ML_OPS.
⠙ Initializing ZenML repository at e:\GitHub\ML_OPS.

⠙ Initializing ZenML repository at e:\GitHub\ML_OPS.


The local active stack was initialized to 'default'. This local configuration 
will only take effect when you're running ZenML from the initialized repository
root, or from a subdirectory. For more information on repositories and 
configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.


In [3]:
import torch
from torch import optim
from pytorch_lightning import LightningModule, Trainer
from torch.utils.data import DataLoader
import torch.nn.functional as F
import mlflow


class ExampleNetwork(LightningModule):
    def __init__(self, model, data_loader, val_data_loader,experimentName):
        super(ExampleNetwork, self).__init__()
        self.mdl: torch.nn.Module = model
        self.data_loader: DataLoader = data_loader
        self.val_data_loader: DataLoader = val_data_loader

        self.lr = 0.001
        self.batch_size = 16

    def forward(self, x: torch.Tensor):
        return self.mdl(x.float())

    def predict(self, x: torch.Tensor):
        with torch.no_grad():
            out = self.forward(x.float())
        return out

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        return self.data_loader

    def val_dataloader(self):
        return self.val_data_loader

    def training_step(self, batch: torch.Tensor, batch_nb: int):
        x, y = batch
        y = torch.squeeze(y.to(torch.int64))
        loss = F.cross_entropy(self(x.float()), y)
        return loss

    def validation_step(self, batch: torch.Tensor, batch_nb: int):
        x, y = batch
        y = torch.squeeze(y.to(torch.int64))
        loss = F.cross_entropy(self(x.float()), y)
        return loss

    def on_fit_end(self):
        mlflow.end_run()

In [7]:
import torch

from torchsig.models.iq_models.efficientnet.efficientnet import efficientnet_b4


model = torch.load("src/classifiers/best_hae.pt")

classes = ["4ask","8pam","16psk","32qam_cross","2fsk","ofdm-256"]
model_eff = efficientnet_b4(
        pretrained=False,
        num_classes=len(classes)
    )

example_model = ExampleNetwork(model_eff,None,None,None)
example_model = example_model.float().to("cuda")

example_model.load_state_dict(torch.load("src/classifiers/10epochs_classifier.pt"))

<All keys matched successfully>

In [8]:

import torchsig
from torch.utils.data import DataLoader
from torchsig.datasets.modulations import ModulationsDataset
import numpy as np
import torch
import torchsig.transforms as ST
from torchsig.utils.cm_plotter import plot_confusion_matrix
from sklearn.metrics import classification_report
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt

In [5]:
iq_samples = 1024
samples_per_class = 1000
batch_size = 32
Hae_epochs = 30
Hqa_epochs = 30
classifier_epochs = 10
trainbool = True
eff_net_PATH = f"./src/classifiers/{classifier_epochs}epochs_classifier.pt"
device = "cuda"
layers = 5
input_feature_dim = 2
enc_hidden_sizes = [16, 16, 32, 64, 128, 256]
dec_hidden_sizes = [16, 64, 256, 512, 1024, 2048]
codebook_slots = 64
codeword_dim = 64
codebook_init = "normal"
batch_norm = 1
reset_choice = 1
num_res_blocks = 0
cos_reset = 1
compress = 2
hae_lr = 4e-4
hqa_lr = 4e-4
hae_Cos_coeff = 0.001
hqa_Cos_coeff = 0.7
KL_coeff = 0.1
CL_coeff = 0.005

accuracies = []
num_recons = 1
num_classes = len(classes)
layers = len(model)
data_transform = ST.Compose([
        ST.ComplexTo2D(),
    ])
ds_test = ModulationsDataset(
        classes = classes,
        use_class_idx = True,
        level=0,
        num_iq_samples=iq_samples,
        num_samples=int(len(classes)*samples_per_class)/10,
        include_snr=False,
        transform = data_transform
    )

num_test_examples = len(ds_test)

for j in range(layers):
    for k in range(num_recons): 
        y_raw_preds = np.empty((num_test_examples, num_classes))
        y_preds = np.zeros((num_test_examples,))
        y_true = np.zeros((num_test_examples,))
        hae = model[j]
        hae = hae.float().to(device)
        hae.eval()
        example_model.to(device).eval()
        for i in tqdm(range(0, num_test_examples)):
            # Retrieve data
            idx = i  # Use index if evaluating over full dataset
            
            data, label = ds_test[idx]
            #test_x = hae.reconstruct(data)
            test_x = hae.reconstruct(torch.from_numpy(np.expand_dims(data, 0)).float().to(device))
            #test_x = torch.from_numpy(np.expand_dims(data, 0)).float().to(device)
            # Infer
            #test_x = torch.from_numpy(np.expand_dims(test_x, 0)).float().to(device)
            pred_tmp = example_model.predict(test_x)
            pred_tmp = pred_tmp.cpu().numpy() if torch.cuda.is_available() else pred_tmp
            # Argmax
            y_preds[i] = np.argmax(pred_tmp)
            # Store label
            y_true[i] = label
    
    
        acc = np.sum(np.asarray(y_preds) == np.asarray(y_true)) / len(y_true)
        plot_confusion_matrix(
            y_true,
            y_preds,
            classes=classes,
            normalize=True,
            title="Example Modulations Confusion Matrix\nTotal Accuracy: {:.2f}%".format(
                acc * 100
            ),
            text=False,
            rotate_x_text=60,
            figsize=(10, 10),
        )
        confusionMatrix_save_path = f"./vis/confusion_matrix_layer_{j+1}_recon_{k+1}.png"
        #plt.savefig(confusionMatrix_save_path)
        print(f"Layer {j+1}\nClassification Report: \nAccuracy {acc*100}")
        print(classification_report(y_true, y_preds))
        matplotlib.pyplot.close()
        accuracies.append(acc*100)

  0%|          | 0/600 [00:00<?, ?it/s]

100%|██████████| 600/600 [00:35<00:00, 16.90it/s]


Layer 1
Classification Report: 
Accuracy 100.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       100
         1.0       1.00      1.00      1.00       100
         2.0       1.00      1.00      1.00       100
         3.0       1.00      1.00      1.00       100
         4.0       1.00      1.00      1.00       100
         5.0       1.00      1.00      1.00       100

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



100%|██████████| 600/600 [00:38<00:00, 15.66it/s]


Layer 2
Classification Report: 
Accuracy 100.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       100
         1.0       1.00      1.00      1.00       100
         2.0       1.00      1.00      1.00       100
         3.0       1.00      1.00      1.00       100
         4.0       1.00      1.00      1.00       100
         5.0       1.00      1.00      1.00       100

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



...

100%|██████████| 600/600 [00:44<00:00, 13.39it/s]


Layer 4
Classification Report: 
Accuracy 93.16666666666666
              precision    recall  f1-score   support

         0.0       1.00      0.91      0.95       100
         1.0       1.00      1.00      1.00       100
         2.0       1.00      0.73      0.84       100
         3.0       0.78      0.95      0.86       100
         4.0       1.00      1.00      1.00       100
         5.0       0.88      1.00      0.93       100

    accuracy                           0.93       600
   macro avg       0.94      0.93      0.93       600
weighted avg       0.94      0.93      0.93       600



100%|██████████| 600/600 [00:46<00:00, 12.93it/s]

Layer 5
Classification Report: 
Accuracy 63.0
              precision    recall  f1-score   support

         0.0       1.00      0.10      0.18       100
         1.0       1.00      1.00      1.00       100
         2.0       0.00      0.00      0.00       100
         3.0       0.40      0.68      0.51       100
         4.0       1.00      1.00      1.00       100
         5.0       0.45      1.00      0.62       100

    accuracy                           0.63       600
   macro avg       0.64      0.63      0.55       600
weighted avg       0.64      0.63      0.55       600




e:\GitHub\ML_OPS\virtualEnv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\GitHub\ML_OPS\virtualEnv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\GitHub\ML_OPS\virtualEnv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
vars(model[3])

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict([(51,
               <function torch.distributed._shard.sharded_tensor.state_dict_hook(module, destination, prefix, local_metadata)>)]),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict([(52,
               <torch.nn.modules.module._WrappedHook at 0x2514f74bd30>)]),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('prev_model',
               HQA(
                 (prev_model): HQA(
                   (prev_model): HQA(
                     (encoder): Encoder(
                       (blocks): Sequential(
    

In [10]:

from zenml.client import Client

artifact = Client().get_artifact_version('1f20bebb-033a-4f60-9dff-0fda94bb9eb4')
loaded_artifact = artifact.load()



In [12]:
vars(loaded_artifact[3])

{'training': False,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('prev_model',
               HAE(
                 (prev_model): HAE(
                   (prev_model): HAE(
                     (encoder): Encoder(
                       (blocks): Sequential(
                         (0): Conv1d(2, 8, kernel_size=(3,), stride=(2,), padding=(1,))
                         (1): Mish()
                         (2): Conv1d(8, 16